In [ ]:
# main.ipynb

# 基础和PyTorch库导入
import os
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np # 引入numpy

# 导入自定义模块
import config
from model import UNet 
from dataset import CamVidDataset, get_class_rgb_values, image_transforms, mask_transforms_for_resize
from trainer import  Trainer, evaluate_model 
from utils import plot_segmentation_results, CLASS_COLORMAP, NUM_CLASSES, CLASS_NAMES, pixel_accuracy, calculate_miou_and_iou_per_class


# 1. 检查设备
print(f"使用设备: {config.DEVICE}")

In [ ]:
# 2. 加载类别信息

# NUM_CLASSES 和 CLASS_COLORMAP 会从 utils.py 中通过 CLASS_CSV_PATH 自动加载
# CLASS_NAMES 也应该从 utils.py 导入，如果它在那里被定义和导出的话
print(f"从 utils.py 加载的信息:")
print(f"  数据集共有 {NUM_CLASSES} 个类别.")
if CLASS_NAMES:
    print(f"  类别名称示例 (来自utils): {CLASS_NAMES[:5]}")
else:
    print("  CLASS_NAMES 未从 utils.py 加载。")

# get_class_rgb_values 用于 Dataset 初始化时显式传递rgb值
# 这个函数是从 dataset.py 导入的
class_rgb_values_for_dataset, class_names_from_dataset = get_class_rgb_values()

if not class_rgb_values_for_dataset:
    print("错误: 无法从 dataset.py 的 get_class_rgb_values 加载类别RGB值。请检查 CSV 文件和路径。")
    CAN_PROCEED = False
else:
    print(f"\n从 dataset.py 的 get_class_rgb_values 加载的信息:")
    print(f"  获取到 {len(class_names_from_dataset)} 个类别名称: {class_names_from_dataset[:5]}...")
    CAN_PROCEED = True

# 确保 utils 和 dataset 加载的类别数量一致
if NUM_CLASSES != len(class_rgb_values_for_dataset) and CAN_PROCEED:
    print(f"警告: utils.py (NUM_CLASSES={NUM_CLASSES}) 和 dataset.py (len={len(class_rgb_values_for_dataset)}) 的类别数量不一致!")
    # 可以选择在这里设置 CAN_PROCEED = False

In [ ]:
# 3. 准备数据集
if CAN_PROCEED:
    print("准备数据集...")
    try:
        train_dataset = CamVidDataset(
            image_dir=os.path.join(config.DATA_DIR, 'train'),
            mask_dir=os.path.join(config.DATA_DIR, 'train_labels'),
            class_rgb_values=class_rgb_values_for_dataset, # 使用从 dataset.py 加载的值
            transform=image_transforms,
            mask_transform=mask_transforms_for_resize
        )

        val_dataset = CamVidDataset(
            image_dir=os.path.join(config.DATA_DIR, 'val'),
            mask_dir=os.path.join(config.DATA_DIR, 'val_labels'),
            class_rgb_values=class_rgb_values_for_dataset, # 使用从 dataset.py 加载的值
            transform=image_transforms,
            mask_transform=mask_transforms_for_resize
        )

        print(f"训练集样本数: {len(train_dataset)}")
        print(f"验证集样本数: {len(val_dataset)}")

        # 可选：测试一个样本
        if len(train_dataset) > 0:
            # 尝试获取一个有效样本
            idx_to_try = 0
            img_sample, mask_sample = None, None
            while idx_to_try < len(train_dataset):
                img_sample, mask_sample = train_dataset[idx_to_try]
                if img_sample is not None and mask_sample is not None:
                    break
                idx_to_try += 1
            
            if img_sample is not None:
                print(f"从训练集获取的图像尺寸: {img_sample.shape}, 掩码尺寸: {mask_sample.shape}, 掩码数据类型: {mask_sample.dtype}")
            else:
                print("无法从训练集获取任何有效样本。")
                CAN_PROCEED = False
        elif len(train_dataset) == 0 : # 如果数据集本身长度为0
             print("训练集为空，无法继续。")
             CAN_PROCEED = False

    except Exception as e:
        print(f"创建数据集时发生错误: {e}")
        CAN_PROCEED = False
else:
    print("由于类别信息加载失败，跳过数据集准备。")

In [ ]:
# 4. 准备DataLoaders

if CAN_PROCEED:
    # 处理Dataset中 __getitem__ 可能返回 (None, None) 的情况
    def collate_fn(batch):
        # 过滤掉那些 __getitem__ 返回 (None, None) 的项
        # batch 是一个列表，每个元素是 (image_tensor, mask_tensor) 或 (None, None)
        batch = list(filter(lambda x: x is not None and x[0] is not None and x[1] is not None, batch))
        if not batch: # 如果过滤后批次为空
            return None, None # Trainer需要能处理这种情况
        return torch.utils.data.dataloader.default_collate(batch)

    train_loader = DataLoader(
        train_dataset,
        batch_size=config.BATCH_SIZE,
        shuffle=True,
        num_workers=config.NUM_WORKERS,
        pin_memory=True if config.DEVICE == "cuda" else False,
        collate_fn=collate_fn
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=config.BATCH_SIZE,
        shuffle=False,
        num_workers=config.NUM_WORKERS,
        pin_memory=True if config.DEVICE == "cuda" else False,
        collate_fn=collate_fn
    )

    print(f"训练 DataLoader 批次数 (估计): {len(train_dataset) // config.BATCH_SIZE}") # 实际长度取决于collate_fn
    print(f"验证 DataLoader 批次数 (估计): {len(val_dataset) // config.BATCH_SIZE}")

    # 检查DataLoader是否能产生数据 (更可靠的检查)
    try:
        train_batch_img, train_batch_mask = next(iter(train_loader))
        if train_batch_img is None:
            print("错误：训练DataLoader返回了空批次。请检查数据集和collate_fn。")
            CAN_PROCEED = False
        else:
            print(f"成功从 train_loader 获取批次: 图像 {train_batch_img.shape}, 掩码 {train_batch_mask.shape}")
    except StopIteration:
        print("错误：训练DataLoader为空。")
        CAN_PROCEED = False
    
    if CAN_PROCEED:
        try:
            val_batch_img, val_batch_mask = next(iter(val_loader))
            if val_batch_img is None:
                print("错误：验证DataLoader返回了空批次。")
                CAN_PROCEED = False
            else:
                print(f"成功从 val_loader 获取批次: 图像 {val_batch_img.shape}, 掩码 {val_batch_mask.shape}")
        except StopIteration:
            print("错误：验证DataLoader为空。")
            CAN_PROCEED = False
            
    if CAN_PROCEED:
        print("DataLoaders准备就绪。")

else:
    print("由于数据集准备失败，跳过DataLoader准备。")

In [ ]:
# 5. 初始化模型
if CAN_PROCEED:
    print("初始化模型...")
    # NUM_CLASSES 是从 utils.py 导入的 (应与dataset.py中的一致)
    model = model = UNet(n_channels=3, n_classes=NUM_CLASSES)
    print(model) # 打印模型结构
    # 将模型移到设备
    model.to(config.DEVICE)
else:
    print("由于前期步骤失败，跳过模型初始化。")

In [ ]:
# 6. 初始化训练器并开始训练
if CAN_PROCEED:
    print("初始化训练器...")
    # 确保 trainer.py 中的 Trainer 类可以处理 DataLoader 返回 (None, None) 的情况
    # 或者确保 collate_fn 不会轻易让这种事情发生给 Trainer
    trainer = Trainer(model, train_loader, val_loader, num_classes=NUM_CLASSES, device=config.DEVICE)

    print("开始训练...")
    # trainer.fit() 应该返回这些列表
    train_losses, val_losses, val_accuracies = trainer.fit(config.NUM_EPOCHS)
else:
    print("由于前期步骤失败，跳过训练。")
    # 定义空列表以便后续单元格不会因变量未定义而报错
    train_losses, val_losses, val_accuracies = [], [], []

In [ ]:
# 7. 绘制训练和验证损失/准确率曲线
if CAN_PROCEED and train_losses and val_losses and val_accuracies: # 确保训练已进行且有数据可画
    plt.figure(figsize=(12, 5)) # 设置图表大小

    # 第一个子图：损失曲线
    plt.subplot(1, 2, 1) # 定义第一个子图 (1行2列中的第1个)
    plt.plot(train_losses, label='Training Loss') # 绘制训练损失曲线
    plt.plot(val_losses, label='Validation Loss') # 绘制验证损失曲线
    plt.xlabel('Epochs') # 设置X轴标签
    plt.ylabel('Loss')   # 设置Y轴标签
    plt.legend()
    plt.title('Loss Curves') # 设置图像标题
    plt.grid(True)

    # 第二个子图：验证准确率曲线
    plt.subplot(1, 2, 2) # 定义第二个子图 (1行2列中的第2个)
    plt.plot(val_accuracies, label='Validation Pixel Accuracy', color='green') # 绘制验证准确率曲线
    plt.xlabel('Epochs') # 设置X轴标签
    plt.ylabel('Accuracy') # 设置Y轴标签
    plt.legend()
    plt.title('Validation Accuracy Curve') # 设置图像标题
    plt.grid(True)

    plt.tight_layout() # 自动调整子图布局，避免重叠
    plt.show()
elif CAN_PROCEED:
    # 这部分 print 语句保留中文，因为它们不是图片上的文字，而是控制台输出
    print("训练已执行，但未能获取到损失或准确率数据用于绘图。")
else:
    print("由于前期步骤失败，没有训练数据可供绘图。")

In [ ]:

# 8. 模型评估 (在验证集上)
# 此单元格使用封装在 trainer.py 中的 evaluate_model 函数进行评估。

# 确保 CAN_PROCEED, config.MODEL_SAVE_PATH 等条件满足
if CAN_PROCEED and config.MODEL_SAVE_PATH and os.path.exists(config.MODEL_SAVE_PATH):
    print(f"\n--- 开始模型评估 (使用封装函数) ---")
    print(f"从 {config.MODEL_SAVE_PATH} 加载模型用于评估...")

    # 1. 重新初始化模型结构并加载权重
    # NUM_CLASSES 应已从 utils.py 正确加载
    eval_model = UNet(n_channels=3, n_classes=NUM_CLASSES)
    try:
        eval_model.load_state_dict(torch.load(config.MODEL_SAVE_PATH, map_location=config.DEVICE))
        eval_model.to(config.DEVICE)
        # model.eval() 会在 evaluate_model 函数内部调用
    except Exception as e:
        print(f"加载模型权重失败: {e}")
        # 如果加载失败，则无法继续评估
        eval_model = None 

    # 2. 准备评估用的DataLoader (通常复用val_loader)
    # 确保 val_loader 在当前作用域内仍然可用且非空
    if 'val_loader' not in globals() or val_loader is None:
        print("错误: val_loader 未定义或为空，无法进行评估。")
        print("请确保之前的单元格已成功运行或考虑在此处重新创建DataLoader。")
        current_eval_loader = None
    else:
        # 检查 DataLoader 是否为空 (如果所有数据都被 collate_fn 过滤掉)
        # 一个简单的方法是尝试获取迭代器长度，但这不适用于所有 DataLoader
        # 更安全的是让 evaluate_model 处理空的 DataLoader
        print("使用已有的 val_loader 进行评估。")
        current_eval_loader = val_loader 

    if eval_model and current_eval_loader:
        # 3. 定义评估时使用的损失函数
        criterion_eval = torch.nn.CrossEntropyLoss()

        # 4. 调用封装的评估函数
        # plot_segmentation_results 来自 utils.py (已导入)
        # CLASS_NAMES 和 NUM_CLASSES 来自 utils.py (已导入)
        # config.DEVICE 来自 config.py (已导入)
        evaluate_model(
            model=eval_model,
            dataloader=current_eval_loader,
            criterion=criterion_eval,
            num_classes=NUM_CLASSES,
            class_names=CLASS_NAMES, # CLASS_NAMES 从 utils 导入
            device=config.DEVICE,
            plot_segmentation_results_func=plot_segmentation_results, # 传递绘图函数
            num_vis_samples=min(3, config.BATCH_SIZE) # 可视化样本数，不超过3或批大小
        )
    elif not eval_model:
        print("由于模型加载失败，跳过评估。")
    else: # current_eval_loader is None
        print("评估加载器 (current_eval_loader) 未准备好或为空，跳过调用 evaluate_model。")

elif not CAN_PROCEED:
    print("由于前期步骤失败，跳过模型评估。")
elif not config.MODEL_SAVE_PATH:
    print("模型保存路径 (MODEL_SAVE_PATH) 未在 config.py 中定义，跳过模型评估。")
elif not os.path.exists(config.MODEL_SAVE_PATH):
    print(f"已保存的模型文件 {config.MODEL_SAVE_PATH} 未找到，跳过模型评估。")
else:
    print("未知原因导致无法进行评估。请检查 CAN_PROCEED 状态和模型路径。")